In [1]:
!pip install -e git+https://github.com/smirnov-am/pyfinmod.git#egg=pyfinmod

Obtaining pyfinmod from git+https://github.com/smirnov-am/pyfinmod.git#egg=pyfinmod
  Cloning https://github.com/smirnov-am/pyfinmod.git to ./src/pyfinmod
  Running command git clone -q https://github.com/smirnov-am/pyfinmod.git '/Users/new/Desktop/FinTech자료/pyfinmod-master (2)/src/pyfinmod'
  Found existing installation: pyfinmod 0.0.1
    Uninstalling pyfinmod-0.0.1:
      Successfully uninstalled pyfinmod-0.0.1
  Running setup.py develop for pyfinmod


## 1. Basic calculations 
### Net present Value (순현재가치)

In [1]:
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
dates = [date.today() + relativedelta(months=i) for i in range(13)]
df = pd.DataFrame(data={'cash flow': [-100] + [10]*12, # 현금흐름은 예시임
                        'date': dates})
annual_discount_rate = 0.1
from pyfinmod.basic import npv
npv(df, annual_discount_rate)


13.986322356973966

### Internal rate of returns 
### (내부수익률_현재가치를 0으로 만들어주는 이자율)

In [2]:
from pyfinmod.basic import irr
dates = [date.today() + relativedelta(years=i) for i in range(6)]
df3 = pd.DataFrame(data={'cash flow': [-145, 100, 100, 100, 100, -275],                              'date': dates})
irr(df3, guess=0.1)

[0.08775586797252909]

### Payment for a loan based on constant payments and a constant interest rate <a name="pmt"></a>

### (대출금을 12개월 분할 납부할 때, 월별 납부액)

In [3]:
from pyfinmod.basic import pmt
pmt(principal=100, annual_interest_rate=0.3, term=12, period='month')



9.57859525723352

### Future value (미래가치)

In [4]:
from pyfinmod.basic import fv
fv(deposits=[1000 for _ in range(10)], annual_interest_rate=0.1,
   period='year')

17531.16706110001

## 2. Continuously compounded interest rate <a name="icc"></a>

In [5]:
df = pd.DataFrame(data={'amount': [1000, 1500], 'date': [date.today(), date.today() + relativedelta(years=1, months=9)]})
from pyfinmod.basic import get_annual_rate_cc
get_annual_rate_cc(df)



0.23169434749037965

## 3. Yahoo Finance parser

In [6]:
from pyfinmod.yahoo_finance import YahooFinanceParser

parser = YahooFinanceParser('000660.KS')
parser.get_dataframe('balance-sheet').head(5)



,2018,2017,2016,2015
row name,,,,
Cash And Cash Equivalents,2349319000000,2949991000000,613786000000,1175719000000
Short Term Investments,5786452000000,5370181000000,3438173000000,3615554000000
Net Receivables,6727532000000,5853397000000,3425345000000,2687787000000
Inventory,4422733000000,2640439000000,2026198000000,1923376000000
Other Current Assets,348046000000,274025000000,96649000000,13155000000


## transpose

In [7]:
import numpy as np
from pyfinmod.yahoo_finance import YahooFinanceParser

parser = YahooFinanceParser('000660.KS')
parser.get_dataframe('balance-sheet')
df = parser.get_dataframe('balance-sheet')

#행열 변환 
df2 = np.transpose(df)


In [8]:
df2

row name,Cash And Cash Equivalents,Short Term Investments,Net Receivables,Inventory,Other Current Assets,Total Current Assets,Long Term Investments,"Property, plant and equipment",Goodwill,Intangible Assets,...,Misc. Stocks Options Warrants,Redeemable Preferred Stock,Preferred Stock,Common Stock,Retained Earnings,Treasury Stock,Capital Surplus,Other Stockholder Equity,Total stockholders' equity,Net Tangible Assets
2018,2349319000000,5786452000000,6727532000000,4422733000000,348046000000,19894146000000,4887778000000,34952617000000,709811000000,815003000000,...,0,0,0,3657652000000,42033601000000,-2989270000000,4143736000000,-480843000000,46845719000000,45320905000000
2017,2949991000000,5370181000000,5853397000000,2640439000000,274025000000,17310444000000,403090000000,24062601000000,695073000000,669967000000,...,0,0,0,3657652000000,27287256000000,-1273364000000,4143736000000,-501451000000,33815280000000,32450240000000
2016,613786000000,3438173000000,3425345000000,2026198000000,96649000000,9838982000000,278795000000,18777402000000,730204000000,555505000000,...,0,0,0,3657652000000,17066583000000,-851016000000,4143736000000,-79103000000,24016955000000,22731246000000
2015,1175719000000,3615554000000,2687787000000,1923376000000,13155000000,9760030000000,254393000000,16966252000000,720755000000,500811000000,...,0,0,0,3657652000000,14358988000000,-773513000000,4143736000000,-1600000000,21386863000000,20165297000000


## 자동 transepose 함수

In [9]:
from pyfinmod.yahoo_finance import YahooFinanceParser
import numpy as np
from pandas import Series, DataFrame


def csl(x):
    parser = YahooFinanceParser(x)
    df=parser.get_dataframe('income-statement')
    #df2=np.transpose(df)
    #df2.insert(0,"companyname",x)
    return(df)

In [10]:
def csl2(x):
    parser = YahooFinanceParser(x)
    df_1=parser.get_dataframe('balance-sheet')
    #df_2=np.transpose(df_1)
    #df_2.insert(0,"companyname",x)
    return(df_1)

df3=csl2('000660.KS')
df3.head()

,2018,2017,2016,2015
row name,,,,
Cash And Cash Equivalents,2349319000000,2949991000000,613786000000,1175719000000
Short Term Investments,5786452000000,5370181000000,3438173000000,3615554000000
Net Receivables,6727532000000,5853397000000,3425345000000,2687787000000
Inventory,4422733000000,2640439000000,2026198000000,1923376000000
Other Current Assets,348046000000,274025000000,96649000000,13155000000


In [11]:
df2.head()

row name,Cash And Cash Equivalents,Short Term Investments,Net Receivables,Inventory,Other Current Assets,Total Current Assets,Long Term Investments,"Property, plant and equipment",Goodwill,Intangible Assets,...,Misc. Stocks Options Warrants,Redeemable Preferred Stock,Preferred Stock,Common Stock,Retained Earnings,Treasury Stock,Capital Surplus,Other Stockholder Equity,Total stockholders' equity,Net Tangible Assets
2018,2349319000000,5786452000000,6727532000000,4422733000000,348046000000,19894146000000,4887778000000,34952617000000,709811000000,815003000000,...,0,0,0,3657652000000,42033601000000,-2989270000000,4143736000000,-480843000000,46845719000000,45320905000000
2017,2949991000000,5370181000000,5853397000000,2640439000000,274025000000,17310444000000,403090000000,24062601000000,695073000000,669967000000,...,0,0,0,3657652000000,27287256000000,-1273364000000,4143736000000,-501451000000,33815280000000,32450240000000
2016,613786000000,3438173000000,3425345000000,2026198000000,96649000000,9838982000000,278795000000,18777402000000,730204000000,555505000000,...,0,0,0,3657652000000,17066583000000,-851016000000,4143736000000,-79103000000,24016955000000,22731246000000
2015,1175719000000,3615554000000,2687787000000,1923376000000,13155000000,9760030000000,254393000000,16966252000000,720755000000,500811000000,...,0,0,0,3657652000000,14358988000000,-773513000000,4143736000000,-1600000000,21386863000000,20165297000000


In [12]:
df3.head()

,2018,2017,2016,2015
row name,,,,
Cash And Cash Equivalents,2349319000000,2949991000000,613786000000,1175719000000
Short Term Investments,5786452000000,5370181000000,3438173000000,3615554000000
Net Receivables,6727532000000,5853397000000,3425345000000,2687787000000
Inventory,4422733000000,2640439000000,2026198000000,1923376000000
Other Current Assets,348046000000,274025000000,96649000000,13155000000


In [14]:
import pandas as pd
df_concat = pd.concat([df2 ,df3])
df_concat.head()

/Users/new/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,Accounts Payable,Accumulated Amortization,Capital Surplus,Cash And Cash Equivalents,Common Stock,Deferred Long Term Asset Charges,Deferred Long Term Liability Charges,Goodwill,Intangible Assets,Inventory,...,Total Assets,Total Current Assets,Total Current Liabilities,Total Liabilities,Total stockholders' equity,Treasury Stock,2018,2017,2016,2015
2018,1.096380e+12,0.0,4.143736e+12,2.349319e+12,3.657652e+12,5.440160e+11,1.153956e+12,7.098110e+11,8.150030e+11,4.422733e+12,...,6.365834e+13,1.989415e+13,1.303185e+13,1.680600e+13,4.684572e+13,-2.989270e+12,NaN,NaN,NaN,NaN
2017,7.585780e+11,0.0,4.143736e+12,2.949991e+12,3.657652e+12,5.997830e+11,8.822500e+11,6.950730e+11,6.699670e+11,2.640439e+12,...,4.541846e+13,1.731044e+13,8.116133e+12,1.159754e+13,3.381528e+13,-1.273364e+12,NaN,NaN,NaN,NaN
2016,6.961440e+11,0.0,4.143736e+12,6.137860e+11,3.657652e+12,7.923680e+11,6.298820e+11,7.302040e+11,5.555050e+11,2.026198e+12,...,3.221603e+13,9.838982e+12,4.160849e+12,8.192496e+12,2.401696e+13,-8.510160e+11,NaN,NaN,NaN,NaN
2015,7.913730e+11,0.0,4.143736e+12,1.175719e+12,3.657652e+12,3.612040e+11,4.833300e+11,7.207550e+11,5.008110e+11,1.923376e+12,...,2.967791e+13,9.760030e+12,4.840698e+12,8.290203e+12,2.138686e+13,-7.735130e+11,NaN,NaN,NaN,NaN
Cash And Cash Equivalents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.349319e+12,2.949991e+12,6.137860e+11,1.175719e+12


In [15]:
df_trans = np.transpose(df_concat)
df_trans.head()

,2018,2017,2016,2015,Cash And Cash Equivalents,Short Term Investments,Net Receivables,Inventory,Other Current Assets,Total Current Assets,...,Misc. Stocks Options Warrants,Redeemable Preferred Stock,Preferred Stock,Common Stock,Retained Earnings,Treasury Stock,Capital Surplus,Other Stockholder Equity,Total stockholders' equity,Net Tangible Assets
Accounts Payable,1.096380e+12,7.585780e+11,6.961440e+11,7.913730e+11,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accumulated Amortization,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Capital Surplus,4.143736e+12,4.143736e+12,4.143736e+12,4.143736e+12,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cash And Cash Equivalents,2.349319e+12,2.949991e+12,6.137860e+11,1.175719e+12,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Common Stock,3.657652e+12,3.657652e+12,3.657652e+12,3.657652e+12,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
def csl3(x):
    df1 = csl(x)
    df2 = csl2(x)
    df_concat11 = pd.concat([df1 ,df2])
    df_trans11 = np.transpose(df_concat11)
    df_trans11.insert(0,"companyname",x)
    return(df_trans11)
csl3('071320.KS')

row name,companyname,Total Revenue,Cost of Revenue,Gross Profit,Research Development,Selling General and Administrative,Non Recurring,Others,Total Operating Expenses,Operating Income or Loss,...,Misc. Stocks Options Warrants,Redeemable Preferred Stock,Preferred Stock,Common Stock,Retained Earnings,Treasury Stock,Capital Surplus,Other Stockholder Equity,Total stockholders' equity,Net Tangible Assets
2018,071320.KS,2487330695000,2374123517000,113207177000,9642562000,80128592000,0,1688091000,2473164696000,14165998000,...,0,0,0,57893720000,1390676928000,-2131618000,113713456000,-2131618000,1560152485000,1481150227000
2017,071320.KS,1834422361000,1625866796000,208555565000,9003795000,71812291000,0,1574215000,1715051434000,119370927000,...,0,0,0,57893720000,1658220879000,-95502000,113713456000,-95502000,1829732552000,1749876025000
2013,071320.KS,2878599260000,2570638509000,307960750000,10048622000,62139766000,0,1110875000,2648615281000,229983978000,...,0,0,0,57893720000,1429921037000,-1506756000,113713456000,-1506756000,1600021456000,1546060695000
2012,071320.KS,2809566616000,2498020325000,311546290000,7500284000,55353329000,0,1043171000,2566657440000,242909175000,...,0,0,0,57893720000,1354094852000,-1896611000,113713456000,-1896611000,1523805416000,1466476867000


### balance와 신용등급(raw)

In [17]:
import pandas as pd
x=['071320.KS','005380.KS','036460.KS','017670.KS','055550.KS','316140.KS','086790.KS','086790.KS',
   '139130.KS','002380.KS','004020.KS','004170.KS','032640.KS','051600.KS','015760.KS','072710.KS','023530.KS','005300.KS',
   '002270.KS','006400.KS','000660.KS','066570.KS','008770.KS','078930.KS','298040.KS','138930.KS','101530.KS','000080.KS',
   '060980.KS','051500.KQ','213500.KS','011780.KS','103140.KS','298040.KS','042670.KS','034020.KS','123040.KQ','011160.KS',
   '068790.KQ','027740.KS','043200.KQ','090370.KS','016380.KS']
y=[]



for i in range(len(x)):
    k2=csl3(x[i]) #추가하려는 애
    if len(x[i]):
        y.append(k2)
    
df5=pd.concat(y)

In [18]:
import pandas as pd
df_test = pd.DataFrame(df5)
df_test.to_csv("testdata2.csv")

In [19]:
datasetb = pd.read_csv("testdata2.csv")
#for i in range(len(dataset)):
datasetb.rename(columns={"Unnamed: 0":"year"}, inplace = True)
datasetb

,year,companyname,Total Revenue,Cost of Revenue,Gross Profit,Research Development,Selling General and Administrative,Non Recurring,Others,Total Operating Expenses,...,Misc. Stocks Options Warrants,Redeemable Preferred Stock,Preferred Stock,Common Stock,Retained Earnings,Treasury Stock,Capital Surplus,Other Stockholder Equity,Total stockholders' equity,Net Tangible Assets
0,2018,071320.KS,2487330695000,2374123517000,113207177000,9642562000,80128592000,0,1688091000,2473164696000,...,0,0,0,57893720000,1390676928000,-2131618000,113713456000,-2131618000,1560152485000,1481150227000
1,2017,071320.KS,1834422361000,1625866796000,208555565000,9003795000,71812291000,0,1574215000,1715051434000,...,0,0,0,57893720000,1658220879000,-95502000,113713456000,-95502000,1829732552000,1749876025000
2,2013,071320.KS,2878599260000,2570638509000,307960750000,10048622000,62139766000,0,1110875000,2648615281000,...,0,0,0,57893720000,1429921037000,-1506756000,113713456000,-1506756000,1600021456000,1546060695000
3,2012,071320.KS,2809566616000,2498020325000,311546290000,7500284000,55353329000,0,1043171000,2566657440000,...,0,0,0,57893720000,1354094852000,-1896611000,113713456000,-1896611000,1523805416000,1466476867000
4,2018,005380.KS,96812609000000,81670479000000,15142130000000,1125603000000,11594362000000,0,0,94390444000000,...,0,0,0,1157982000000,66490082000000,-4206320000000,4201214000000,-3051076000000,67642958000000,66505918000000
5,2017,005380.KS,96376079000000,78798172000000,17577907000000,1039260000000,11963980000000,0,0,91801412000000,...,0,0,0,1157982000000,67332328000000,-3919051000000,4201214000000,-2278955000000,68772473000000,67545251000000
6,2016,005380.KS,93649024000000,75959720000000,17689304000000,1019188000000,11476616000000,0,0,88455524000000,...,0,0,0,1157982000000,64361408000000,-2863340000000,4202597000000,-1223244000000,66858647000000,65603465000000
7,2015,005380.KS,91958736000000,73701296000000,18257440000000,929280000000,10970254000000,0,0,85600830000000,...,0,0,0,1157982000000,60035088000000,-3020518000000,3520395000000,-1431821000000,61692947000000,60410641000000
8,2018,036460.KS,26185037650000,24510879509000,1674158141000,60409501000,230047623000,0,69312068000,24909162593000,...,0,0,0,461565000000,5497914458000,908442243000,1324901254000,1010865578000,8192822956000,6420995695000
9,2017,036460.KS,22172167684000,20732059573000,1440108111000,55363704000,232048798000,0,95475897000,21135649619000,...,0,0,0,461565000000,4601193102000,1425679844000,1324901254000,1528103178000,7813339200000,5999442759000


In [23]:
df10=pd.read_csv("yearfinal.csv")
#credit은 데이터프레임2
credit=df10.dropna()
credit.shape

(190, 3)

In [48]:
dfaa=pd.read_csv("year2.csv")
#credit은 데이터프레임2
dfaa=dfaa.dropna()
dfaa.head()

,year,companyname,value
0,2015,071320.KS,AAA
1,2015,005380.KS,AAA
2,2015,030200.KS,AAA
3,2015,036460.KS,AAA
4,2015,017670.KS,AAA


In [24]:
df_merge=pd.merge(datasetb,credit[['year','companyname','value']],
                  how='left',
                  on=['year', 'companyname'])

In [25]:
Finalaa=df_merge.dropna()
Finalaa=Finalaa.drop_duplicates()
Finalaa.head()

,year,companyname,Total Revenue,Cost of Revenue,Gross Profit,Research Development,Selling General and Administrative,Non Recurring,Others,Total Operating Expenses,...,Redeemable Preferred Stock,Preferred Stock,Common Stock,Retained Earnings,Treasury Stock,Capital Surplus,Other Stockholder Equity,Total stockholders' equity,Net Tangible Assets,value
0,2018,071320.KS,2487330695000,2374123517000,113207177000,9642562000,80128592000,0,1688091000,2473164696000,...,0,0,57893720000,1390676928000,-2131618000,113713456000,-2131618000,1560152485000,1481150227000,700.0
1,2017,071320.KS,1834422361000,1625866796000,208555565000,9003795000,71812291000,0,1574215000,1715051434000,...,0,0,57893720000,1658220879000,-95502000,113713456000,-95502000,1829732552000,1749876025000,700.0
4,2018,005380.KS,96812609000000,81670479000000,15142130000000,1125603000000,11594362000000,0,0,94390444000000,...,0,0,1157982000000,66490082000000,-4206320000000,4201214000000,-3051076000000,67642958000000,66505918000000,700.0
5,2017,005380.KS,96376079000000,78798172000000,17577907000000,1039260000000,11963980000000,0,0,91801412000000,...,0,0,1157982000000,67332328000000,-3919051000000,4201214000000,-2278955000000,68772473000000,67545251000000,700.0
6,2016,005380.KS,93649024000000,75959720000000,17689304000000,1019188000000,11476616000000,0,0,88455524000000,...,0,0,1157982000000,64361408000000,-2863340000000,4202597000000,-1223244000000,66858647000000,65603465000000,700.0


In [26]:
Finalaa.to_csv("testscl.csv")

## 독립적인 정보 170개(income)

170

2487330695000

# 최종 합친 dataframe

In [27]:
Final=df_merge.dropna()

In [28]:
Final.drop_duplicates()

,year,companyname,Total Revenue,Cost of Revenue,Gross Profit,Research Development,Selling General and Administrative,Non Recurring,Others,Total Operating Expenses,...,Redeemable Preferred Stock,Preferred Stock,Common Stock,Retained Earnings,Treasury Stock,Capital Surplus,Other Stockholder Equity,Total stockholders' equity,Net Tangible Assets,value
0,2018,071320.KS,2487330695000,2374123517000,113207177000,9642562000,80128592000,0,1688091000,2473164696000,...,0,0,57893720000,1390676928000,-2131618000,113713456000,-2131618000,1560152485000,1481150227000,700.0
1,2017,071320.KS,1834422361000,1625866796000,208555565000,9003795000,71812291000,0,1574215000,1715051434000,...,0,0,57893720000,1658220879000,-95502000,113713456000,-95502000,1829732552000,1749876025000,700.0
4,2018,005380.KS,96812609000000,81670479000000,15142130000000,1125603000000,11594362000000,0,0,94390444000000,...,0,0,1157982000000,66490082000000,-4206320000000,4201214000000,-3051076000000,67642958000000,66505918000000,700.0
5,2017,005380.KS,96376079000000,78798172000000,17577907000000,1039260000000,11963980000000,0,0,91801412000000,...,0,0,1157982000000,67332328000000,-3919051000000,4201214000000,-2278955000000,68772473000000,67545251000000,700.0
6,2016,005380.KS,93649024000000,75959720000000,17689304000000,1019188000000,11476616000000,0,0,88455524000000,...,0,0,1157982000000,64361408000000,-2863340000000,4202597000000,-1223244000000,66858647000000,65603465000000,700.0
7,2015,005380.KS,91958736000000,73701296000000,18257440000000,929280000000,10970254000000,0,0,85600830000000,...,0,0,1157982000000,60035088000000,-3020518000000,3520395000000,-1431821000000,61692947000000,60410641000000,700.0
8,2018,036460.KS,26185037650000,24510879509000,1674158141000,60409501000,230047623000,0,69312068000,24909162593000,...,0,0,461565000000,5497914458000,908442243000,1324901254000,1010865578000,8192822956000,6420995695000,700.0
9,2017,036460.KS,22172167684000,20732059573000,1440108111000,55363704000,232048798000,0,95475897000,21135649619000,...,0,0,461565000000,4601193102000,1425679844000,1324901254000,1528103178000,7813339200000,5999442759000,700.0
10,2016,036460.KS,21108115739000,19708507639000,1399608099000,46637857000,209286082000,0,120648501000,20109936699000,...,0,0,461565000000,5811331986000,1762480162000,1324901254000,1864903496000,9360278403000,6944602188000,700.0
11,2015,036460.KS,26052723827000,24672731229000,1379992598000,50883649000,200492639000,0,86291763000,25044911729000,...,0,0,461565000000,6418998526000,1851478788000,1324901254000,1953902122000,10056943569000,7536114053000,700.0


## 

## 4. Enterprise value

### Free Cash Flows (잉여 현금흐름)

In [18]:
from pyfinmod.yahoo_finance import YahooFinanceParser
from pyfinmod.ev import get_enterprise_value

from pyfinmod.ev import _ev

parser = YahooFinanceParser('000660.KS')
df = parser.get_dataframe('balance-sheet')
get_enterprise_value(df)

2018-12-31    57736765000000
2017-12-31    40166421000000
2016-12-31    30194339000000
2015-12-31    26561695000000
dtype: int64

In [19]:
from pyfinmod.ev import get_fcf_from_cscf

parser = YahooFinanceParser('000660.KS')


beta = parser.get_value('Beta (3Y Monthly)')
income_statement = parser.get_dataframe('income-statement')

aapl_parser = YahooFinanceParser('000660.KS')
balance_sheet = aapl_parser.get_dataframe('balance-sheet')
cash_flow = aapl_parser.get_dataframe('cash-flow')
get_fcf_from_cscf(income_statement, cash_flow)
FCF=get_fcf_from_cscf(income_statement, cash_flow)

print(FCF)

2018-12-31    8.673609e+11
2017-12-31    2.869576e+12
2016-12-31   -5.709865e+11
2015-12-31    2.291225e+12
dtype: float64


## 5. Weighted average cost of capital and DCF

In [20]:
from pyfinmod.yahoo_finance import YahooFinanceParser
from pyfinmod.wacc import wacc
from pyfinmod.ev import dcf

parser = YahooFinanceParser('000660.KS')
e = parser.get_value('Market Cap')
beta = parser.get_value('Beta (3Y Monthly)')
income_statement = parser.get_dataframe('income-statement')
balance_sheet = parser.get_dataframe('balance-sheet')

WACC= wacc(e, balance_sheet, income_statement, beta,
                 risk_free_interest_rate=0.02,
                 market_return=0.08)
DCF=dcf(FCF, WACC, short_term_growth=0.08, long_term_growth=0.04)

print(WACC)
print()
print()
print(DCF)
print()
print()
print(FCF)

0.06766361618707539


43052533187841.48


2018-12-31    8.673609e+11
2017-12-31    2.869576e+12
2016-12-31   -5.709865e+11
2015-12-31    2.291225e+12
dtype: float64


###  DF 3개..concating

In [29]:
def csl4(x):
    parser = YahooFinanceParser(x)
    df_1=parser.get_dataframe('cash-flow')
    #df_2=np.transpose(df_1)
    #df_2.insert(0,"companyname",x)
    return(df_1)

df3=csl4('000660.KS')
df3.head()

,2018,2017,2016,2015
row name,,,,
Net Income,15540111000000,10641512000000,2953774000000,4322356000000
Depreciation,6093569000000,4766956000000,4251257000000,3953236000000
Adjustments To Net Income,2255671000000,2213003000000,-402261000000,211645000000
Changes In Accounts Receivables,-547255000000,-2964272000000,-470792000000,1260172000000
Changes In Liabilities,58773000000,514751000000,-208439000000,-156074000000


In [30]:
def cslking(x):
    df1 = csl(x)
    df2 = csl2(x)
    df3 = csl4(x)
    df_concat11 = pd.concat([df1 ,df2,df3])
    df_trans11 = np.transpose(df_concat11)
    df_trans11.insert(0,"companyname",x)
    return(df_trans11)
cslking('001520.KS')

row name,companyname,Total Revenue,Cost of Revenue,Gross Profit,Research Development,Selling General and Administrative,Non Recurring,Others,Total Operating Expenses,Operating Income or Loss,...,Investments,Other Cash flows from Investing Activities,Total Cash Flows From Investing Activities,Dividends Paid,Sale Purchase of Stock,Net Borrowings,Other Cash Flows from Financing Activities,Total Cash Flows From Financing Activities,Effect Of Exchange Rate Changes,Change In Cash and Cash Equivalents
2018,001520.KS,529745192000,472857681000,56887510000,959000000,48495368000,0,2931000000,529028049000,717142000,...,29289050000,-5538716000,523101000,-10372542000,0,316879000,350948000,-13929923000,214504000,10623315000
2017,001520.KS,483087799000,427980316000,55107482000,703000000,43736681000,0,2231000000,476287997000,6799801000,...,50466829000,414985000,17131349000,-10608332000,0,-25194000,217941000,-22071035000,-1008107000,-13740855000
2016,001520.KS,439901518000,387474619000,52426898000,117000000,40925457000,0,2167000000,432249077000,7652441000,...,84753619000,-371482000,37719280000,-24616349000,0,-2609509000,-40135000,-126885260000,331679000,-220826517000
2015,001520.KS,392665078000,325001629000,67663448000,117000000,29824752000,0,1564000000,357532382000,35132695000,...,-334816341000,786660323000,453533203000,-24616349000,0,-2112769000,-292389574000,-291808009000,-48955000,199243638000


In [20]:
x[51]

'036630.KS'

In [21]:
import pandas as pd
x=['071320.KS','005380.KS','036460.KS','017670.KS','055550.KS','316140.KS','086790.KS','086790.KS',
   '139130.KS','002380.KS','004020.KS','004170.KS','032640.KS','051600.KS','015760.KS','072710.KS','023530.KS','005300.KS',
   '002270.KS','006400.KS','000660.KS','066570.KS','008770.KS','078930.KS','298040.KS','138930.KS','101530.KS','000080.KS',
   '060980.KS','051500.KQ','213500.KS','011780.KS','103140.KS','298040.KS','042670.KS','034020.KS','123040.KQ','011160.KS',
   '068790.KQ','027740.KS','043200.KQ','090370.KS','016380.KS','001020.KS','032980.KQ','042660.KS','128820.KS',
   '011200.KS','011930.KS','018680.KQ','007980.KS','036630.KQ','001440.KS','014790.KS','019170.KS','004540.KS','003490.KS',
   '073240.KS','036420.KQ','020560.KS','008040.KS','001230.KS','001520.KS']
y=[]

#'001020.KS','032980.KQ','042660.KS','128820.KS',
#'011200.KS','011930.KS','018680.KQ','007980.KS','036630.KQ','001440.KS','014790.KS','019170.KS','004540.KS','003490.KS',
#'073240.KS','036420.KQ','020560.KS','008040.KS','001230.KS','001520.KS'


for i in range(len(x)):
    print(i)
    k2=cslking(x[i]) #추가하려는 애
    if len(x[i]):
        y.append(k2)
    
df5=pd.concat(y)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62


In [23]:
df5

row name,companyname,Total Revenue,Cost of Revenue,Gross Profit,Research Development,Selling General and Administrative,Non Recurring,Others,Total Operating Expenses,Operating Income or Loss,...,Investments,Other Cash flows from Investing Activities,Total Cash Flows From Investing Activities,Dividends Paid,Sale Purchase of Stock,Net Borrowings,Other Cash Flows from Financing Activities,Total Cash Flows From Financing Activities,Effect Of Exchange Rate Changes,Change In Cash and Cash Equivalents
2018,071320.KS,2487330695000,2374123517000,113207177000,9642562000,80128592000,0,1688091000,2473164696000,14165998000,...,-23958000000,1477780000,-434300755000,-32652058000,0,59818009000,-705614000,26460336000,0,-25425812000
2017,071320.KS,1834422361000,1625866796000,208555565000,9003795000,71812291000,0,1574215000,1715051434000,119370927000,...,-15000000000,-3293082000,-651888338000,-43999227000,0,279806114000,-1215443000,234591444000,0,28115191000
2013,071320.KS,2878599260000,2570638509000,307960750000,10048622000,62139766000,0,1110875000,2648615281000,229983978000,...,-16791838000,-19999000,-401331523000,-43420290000,0,-38369110000,-972979000,-82762379000,0,-35868000
2012,071320.KS,2809566616000,2498020325000,311546290000,7500284000,55353329000,0,1043171000,2566657440000,242909175000,...,37595362000,-30000000,-346385810000,-4052560000,0,43646387000,-564191000,39029635000,0,34497664000
2018,005380.KS,96812609000000,81670479000000,15142130000000,1125603000000,11594362000000,0,0,94390444000000,2422165000000,...,2302476000000,26556000000,-2415064000000,-1127452000000,0,500941000000,200463000000,-880782000000,-79273000000,292096000000
2017,005380.KS,96376079000000,78798172000000,17577907000000,1039260000000,11963980000000,0,0,91801412000000,4574667000000,...,-381440000000,36568000000,-4744413000000,-1138661000000,0,3215832000000,104020000000,2181191000000,-427759000000,931440000000
2016,005380.KS,93649024000000,75959720000000,17689304000000,1019188000000,11476616000000,0,0,88455524000000,5193500000000,...,-2078757000000,2991000000,-6311750000000,-1084546000000,0,7124760000000,-87244000000,5691418000000,181994000000,558626000000
2015,005380.KS,91958736000000,73701296000000,18257440000000,929280000000,10970254000000,0,0,85600830000000,6357906000000,...,1193819000000,15495000000,-8060288000000,-1352510000000,0,8876988000000,4134000000,7213667000000,-166844000000,234950000000
2018,036460.KS,26185037650000,24510879509000,1674158141000,60409501000,230047623000,0,69312068000,24909162593000,1275875057000,...,2717629000,-68798157000,-1275174704000,-8842561000,0,1045039137000,-41996384000,994200190000,22378703000,-201509314000
2017,036460.KS,22172167684000,20732059573000,1440108111000,55363704000,232048798000,0,95475897000,21135649619000,1036518065000,...,117908517000,30124527000,-1225943868000,-7134543000,0,-1468316779000,209324771000,-1266126550000,-57733733000,-42206190000


In [24]:
import pandas as pd
#df_test = pd.DataFrame(df5)
#df_test.to_csv("testking.csv")
dfttt = pd.DataFrame(df5)
dfttt.to_csv("test20.csv")

In [25]:
d1 = pd.read_csv("test20.csv")
#for i in range(len(dataset)):
d1.rename(columns={"Unnamed: 0":"year"}, inplace = True)
d1.head()

,year,companyname,Total Revenue,Cost of Revenue,Gross Profit,Research Development,Selling General and Administrative,Non Recurring,Others,Total Operating Expenses,...,Investments,Other Cash flows from Investing Activities,Total Cash Flows From Investing Activities,Dividends Paid,Sale Purchase of Stock,Net Borrowings,Other Cash Flows from Financing Activities,Total Cash Flows From Financing Activities,Effect Of Exchange Rate Changes,Change In Cash and Cash Equivalents
0,2018,071320.KS,2487330695000,2374123517000,113207177000,9642562000,80128592000,0,1688091000,2473164696000,...,-23958000000,1477780000,-434300755000,-32652058000,0,59818009000,-705614000,26460336000,0,-25425812000
1,2017,071320.KS,1834422361000,1625866796000,208555565000,9003795000,71812291000,0,1574215000,1715051434000,...,-15000000000,-3293082000,-651888338000,-43999227000,0,279806114000,-1215443000,234591444000,0,28115191000
2,2013,071320.KS,2878599260000,2570638509000,307960750000,10048622000,62139766000,0,1110875000,2648615281000,...,-16791838000,-19999000,-401331523000,-43420290000,0,-38369110000,-972979000,-82762379000,0,-35868000
3,2012,071320.KS,2809566616000,2498020325000,311546290000,7500284000,55353329000,0,1043171000,2566657440000,...,37595362000,-30000000,-346385810000,-4052560000,0,43646387000,-564191000,39029635000,0,34497664000
4,2018,005380.KS,96812609000000,81670479000000,15142130000000,1125603000000,11594362000000,0,0,94390444000000,...,2302476000000,26556000000,-2415064000000,-1127452000000,0,500941000000,200463000000,-880782000000,-79273000000,292096000000


In [28]:
df10=pd.read_csv("yearfinal.csv")
#credit은 데이터프레임2
credit=df10.dropna()
credit.shape

(190, 3)

In [29]:
dfking=pd.merge(d1,credit[['year','companyname','value']],
                  how='left',
                  on=['year', 'companyname'])
dfking.shape

(253, 79)

In [30]:
dfking=dfking.dropna()
dfking=dfking.drop_duplicates()
dfking.head()

,year,companyname,Total Revenue,Cost of Revenue,Gross Profit,Research Development,Selling General and Administrative,Non Recurring,Others,Total Operating Expenses,...,Other Cash flows from Investing Activities,Total Cash Flows From Investing Activities,Dividends Paid,Sale Purchase of Stock,Net Borrowings,Other Cash Flows from Financing Activities,Total Cash Flows From Financing Activities,Effect Of Exchange Rate Changes,Change In Cash and Cash Equivalents,value
0,2018,071320.KS,2487330695000,2374123517000,113207177000,9642562000,80128592000,0,1688091000,2473164696000,...,1477780000,-434300755000,-32652058000,0,59818009000,-705614000,26460336000,0,-25425812000,700.0
1,2017,071320.KS,1834422361000,1625866796000,208555565000,9003795000,71812291000,0,1574215000,1715051434000,...,-3293082000,-651888338000,-43999227000,0,279806114000,-1215443000,234591444000,0,28115191000,700.0
4,2018,005380.KS,96812609000000,81670479000000,15142130000000,1125603000000,11594362000000,0,0,94390444000000,...,26556000000,-2415064000000,-1127452000000,0,500941000000,200463000000,-880782000000,-79273000000,292096000000,700.0
5,2017,005380.KS,96376079000000,78798172000000,17577907000000,1039260000000,11963980000000,0,0,91801412000000,...,36568000000,-4744413000000,-1138661000000,0,3215832000000,104020000000,2181191000000,-427759000000,931440000000,700.0
6,2016,005380.KS,93649024000000,75959720000000,17689304000000,1019188000000,11476616000000,0,0,88455524000000,...,2991000000,-6311750000000,-1084546000000,0,7124760000000,-87244000000,5691418000000,181994000000,558626000000,700.0


In [31]:
import pandas as pd
dfking.to_csv("csl20.csv", mode='w')